<a href="https://colab.research.google.com/github/Prathamesh001/openseespy-cantilever-study/blob/main/QuadrilateralPlaneStress/Cantillever_Quad_element_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install openseespy
!pip install opsvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 2.2 MB/s eta 0:00:00


# **Working CODE**

In [6]:
no_of_trials = 10
nx = 30        # Elements along length
ny = 1         # Elements along height
import os
import openseespy.opensees as ops
import opsvis as opsv
import matplotlib.pyplot as plt
import numpy as np

iteration_numbers = []
deflections = []
bending_moments = []
max_stresses = []
for itr_no in range(1, no_of_trials+1):

  # -----------------------------
  # Parameters
  # -----------------------------
  L = 3.0        # Length (m)
  H = 0.1        # Height (m)
  t = 0.1        # Thickness (m)
  E = 210e9       # Elastic modulus (Pa)
  nu = 0.2       # Poisson's ratio
  rho = 2500     # Density (kg/m³)
  g = 9.81       # Gravity (m/s²)

  # -----------------------------
  # Initialize model
  # -----------------------------
  ops.wipe()
  ops.model('basic', '-ndm', 2, '-ndf', 2)

  # -----------------------------
  # Generate nodes
  # -----------------------------
  dx = L / nx
  dy = H / ny

  nodeTag = 1
  node_map = {}
  #x_coords = []
  #y_coords = []
  for i in range(nx + 1):
      for j in range(ny + 1):
          x = i * dx
          #x_coords.append(x)
          y = j * dy
          #y_coords.append(y)
          ops.node(nodeTag, x, y)
          node_map[(i, j)] = nodeTag
          #print(node_map[(i, j)])
          nodeTag += 1
  #print(x_coords)
  #print(y_coords)
  # -----------------------------
  # Fix left edge (cantilever root)
  # -----------------------------
  for j in range(ny + 1):
      n = node_map[(0, j)]
      #print(n)
      ops.fix(n, 1, 1)

  # -----------------------------
  # Material definition
  # -----------------------------
  ops.nDMaterial('ElasticIsotropic', 1, E, nu)

  # -----------------------------
  # Elements
  # -----------------------------
  eleTag = 1
  for i in range(nx):
      for j in range(ny):
          n1 = node_map[(i, j)]
          n2 = node_map[(i + 1, j)]
          n3 = node_map[(i + 1, j + 1)]
          n4 = node_map[(i, j + 1)]
          #print(f"{n1}, {n2}, {n3}, {n4}")
          ops.element('quad', eleTag, n1, n2, n3, n4, t, 'PlaneStress', 1)
          eleTag += 1
  #print(nodeTag-1-ny) #the last bottom node on the free end
  #print(ops.nodeCoord(nodeTag-1-ny))
  # -----------------------------
  # Loading: Point load at free end top
  # -----------------------------
  ops.timeSeries('Linear', 1)
  ops.pattern('Plain', 1, 1)

  ops.load(nodeTag-1, 0, -100) #nodeTag-1 is free end top node
  # -----------------------------
  # Analysis
  # -----------------------------
  ops.system('BandGeneral')
  ops.numberer('RCM')
  ops.constraints('Plain')
  ops.integrator('LoadControl', 1.0)
  ops.algorithm('Linear')
  ops.analysis('Static')
  ops.analyze(1)

  # -----------------------------
  # Plot deformed shape
  # -----------------------------
  fig1, ax1 = plt.subplots(figsize=(40, 8))
  opsv.plot_model(node_labels=1, element_labels=0, ax = ax1)
  plt.title("Cantilever Beam Node")
  #plt.show()

  fig2, ax2 = plt.subplots(figsize=(40, 8))
  opsv.plot_model(node_labels=0, element_labels=1, ax = ax2)
  plt.title("Cantilever Beam Elements")
  #plt.show()

  fig3, ax3 = plt.subplots(figsize=(40, 8))
  opsv.plot_defo(ax = ax3)
  plt.title("Deformed Shape")
  #plt.show()


  print(ops.nodeDisp(nodeTag-1-ny)) # Free end bottom node

  # Get all node and element tags
  node_tags = ops.getNodeTags()
  ele_tags = ops.getEleTags()

  # 1. Build node-to-element map
  node_to_elements = {n: [] for n in node_tags}
  for e in ele_tags:
      nodes = ops.eleNodes(e)
      for n in nodes:
          node_to_elements[n].append(e)

  # 2. Compute average stress per node (with Gauss point averaging)
  nodal_stresses = {}
  for n in node_tags:
      connected_elements = node_to_elements[n]
      if not connected_elements:
          continue
      sum_sigma = np.zeros(3)
      for e in connected_elements:
          stress_raw = np.array(ops.eleResponse(e, 'stress'))
          if len(stress_raw) == 3:
              # Single point stress
              stress_avg = stress_raw
          else:
              # Multiple Gauss points (e.g. 4 for quad)
              stress_ip = stress_raw.reshape((-1, 3))
              stress_avg = stress_ip.mean(axis=0)
          sum_sigma += stress_avg
      nodal_stresses[n] = sum_sigma / len(connected_elements)


  # 3. For nodes 1-9: extract σxx and y-coordinate
  selected_nodes = list(range(1, ny+2))
  node_y = []
  node_sigma_xx = []

  for n in selected_nodes:
      if n in nodal_stresses:
          stress = nodal_stresses[n]
          sigma_xx = stress[0]
          y_coord = ops.nodeCoord(n)[1]
          node_sigma_xx.append(sigma_xx)
          node_y.append(y_coord)
      else:
          print(f"Warning: No stress computed for node {n}")

  # 4. Plot σxx vs y-coordinate
  fig4, ax4 = plt.subplots(figsize=(12, 12))
  ax4.plot(node_sigma_xx, node_y, 'o-', color='blue')
  plt.xlabel("σₓₓ (Pa)")
  plt.ylabel("Node Y-coordinate (m)")
  plt.title("σₓₓ vs Y for Support Cross Section")
  plt.grid(True)
  #plt.show()

  # Define your vertical section nodes (assumed at x=0)
  section_nodes = list(range(1, ny+2))   # Nodes 1 to 9

  # Gather Y-coordinates and sigma_xx for those nodes
  node_y = []
  node_sigma_xx = []

  for n in section_nodes:
      y_coord = ops.nodeCoord(n)[1]
      if n in nodal_stresses:
          sigma_xx = nodal_stresses[n][0]
      else:
          sigma_xx = 0.0
          print(f"Warning: No stress for node {n}")
      node_y.append(y_coord)
      node_sigma_xx.append(sigma_xx)

  # Sort nodes by Y
  data = sorted(zip(node_y, node_sigma_xx))
  y_sorted, sigma_sorted = zip(*data)

  # Integrate using trapezoidal rule
  axial_force = 0.0
  bending_moment = 0.0

  for i in range(len(y_sorted)-1):
      y1, y2 = y_sorted[i], y_sorted[i+1]
      s1, s2 = sigma_sorted[i], sigma_sorted[i+1]

      dy = y2 - y1
      sigma_avg = 0.5 * (s1 + s2)
      y_avg = 0.5 * (y1 + y2)

      dA = t * dy
      axial_force += sigma_avg * dA
      bending_moment += sigma_avg * y_avg * dA

  # Print results
  #print(f"Axial Force at nodes 1-9 section: {axial_force:.2f} N")
  #print(f"Bending Moment at nodes 1-9 section: {bending_moment:.2f} Nm")

  bending_moment_values = []
  x_locations = []

  for i in range(nx+1):
      x_section = i * dx
      section_nodes = [n for n in node_tags if abs(ops.nodeCoord(n)[0] - x_section) < 1e-6]

      if not section_nodes:
          continue

      y_coords = []
      sigma_xx_list = []

      for n in section_nodes:
          y_coords.append(ops.nodeCoord(n)[1])
          sigma_xx_list.append(nodal_stresses.get(n, np.zeros(3))[0])

      # Sort by y
      data = sorted(zip(y_coords, sigma_xx_list))
      y_sorted, sigma_sorted = zip(*data)

      # Integrate
      axial_force = 0.0
      bending_moment = 0.0

      for j in range(len(y_sorted)-1):
          dy = y_sorted[j+1] - y_sorted[j]
          sigma_avg = 0.5 * (sigma_sorted[j] + sigma_sorted[j+1])
          y_avg = 0.5 * (y_sorted[j] + y_sorted[j+1])
          dA = t * dy
          axial_force += sigma_avg * dA
          bending_moment += sigma_avg * y_avg * dA

      bending_moment_values.append(bending_moment)
      x_locations.append(x_section)

  # Now plot
  fig5, ax5 = plt.subplots(figsize=(12, 12))
  ax5.plot(x_locations, bending_moment_values, 'o-', color = 'blue')
  plt.xlabel("X (m)")
  plt.ylabel("Bending Moment (Nm)")
  plt.title("Bending Moment Diagram")
  plt.grid()
  #plt.show()


  iteration_numbers.append(itr_no)
  free_end_deflection = (ops.nodeDisp(nodeTag-1-ny)[1])
  deflections.append(free_end_deflection)
  bending_moments.append(max(bending_moment_values))
  max_stresses.append(max(node_sigma_xx))

  print("\nSummary Table:")
  print(f"{'Iteration':<10}{'Deflection (m)':<20}{'Bending Moment (Nm)':<25}{'Max Stress (Pa)':<20}")
  for i in range(len(iteration_numbers)):
    print(f"{iteration_numbers[i]:<10}{deflections[i]:<20.6e}{bending_moments[i]:<25.6e}{max_stresses[i]:<20.6e}")

  #folder = f'drive/MyDrive/Cantillever_2D_Diagrams/Iteration_{itr_no}'
  #os.makedirs(folder, exist_ok=True)
  #fig1.savefig(os.path.join(folder, f'Node_nos_itr_{itr_no}_nx_{nx}_ny_{ny}.pdf'))
  #fig2.savefig(os.path.join(folder, f'Element_nos_itr_{itr_no}_nx_{nx}_ny_{ny}.pdf'))
  #fig3.savefig(os.path.join(folder, f'Defo_itr_{itr_no}_nx_{nx}_ny_{ny}.pdf'))
  #fig4.savefig(os.path.join(folder, f'Stress_itr_{itr_no}_nx_{nx}_ny_{ny}.pdf'))
  #fig5.savefig(os.path.join(folder, f'BM_itr_{itr_no}_nx_{nx}_ny_{ny}.pdf'))

  ny = ny+1         # Elements along height
  nx = 30*ny        # Elements along length
  plt.close('all')
  print("Iteration number", itr_no, "complete.")

[-8.815850340641375e-06, -0.00035289454426605456]

Summary Table:
Iteration Deflection (m)      Bending Moment (Nm)      Max Stress (Pa)     
1         -3.528945e-04       4.105095e-11             8.044299e-08        
Iteration number 1 complete.
[-1.153003964644355e-05, -0.0004615052515800319]

Summary Table:
Iteration Deflection (m)      Bending Moment (Nm)      Max Stress (Pa)     
1         -3.528945e-04       4.105095e-11             8.044299e-08        
2         -4.615053e-04       1.006410e+02             8.051283e+05        
Iteration number 2 complete.
[-1.2228770759198033e-05, -0.0004895025314617882]

Summary Table:
Iteration Deflection (m)      Bending Moment (Nm)      Max Stress (Pa)     
1         -3.528945e-04       4.105095e-11             8.044299e-08        
2         -4.615053e-04       1.006410e+02             8.051283e+05        
3         -4.895025e-04       1.894880e+02             1.136928e+06        
Iteration number 3 complete.
[-1.2494031745108896e-05, -0.000